<a href="https://colab.research.google.com/github/branjbar/nlp-bert-family/blob/master/tensorflow/tensorflow_roberta_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# https://www.kaggle.com/rftexas/roberta-with-tensorflow-on-tpu/comments

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install transformers
import os
import pandas as pd, numpy as np
import time, re, math
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold
from transformers import *
import tokenizers
print('TF version',tf.__version__)

TF version 2.2.0-rc3


In [4]:
# Detect hardware
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError: # If TPU not found
  tpu = None

# Select appropriate distribution strategy
if tpu:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    tf.config.experimental_connect_to_cluster(resolver)
    # This is the TPU initialization code that has to be at the beginning.
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.experimental.TPUStrategy(resolver)

    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])  
else:
    strategy = tf.distribute.get_strategy() # Default strategy that works on CPU and single GPU
    print('Running on CPU instead')
print("Number of accelerators: ", strategy.num_replicas_in_sync)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.51.87.242:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.51.87.242:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU  ['10.51.87.242:8470']
Number of accelerators:  8


# Initialization

In [0]:
KAGGLE_INPUT_PATH = "../input/"
COLAB_INPUT_PATH = "/content/drive/My Drive/input/"
INPUT_PATH = COLAB_INPUT_PATH

DATA_PATH = INPUT_PATH + "tweet-sentiment-extraction/"
CHECKPOINT_PATH = INPUT_PATH + "model_checkpoint/roberta/"
ROBERTA_PATH = INPUT_PATH + 'tf-roberta/'

In [0]:
tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab_file= ROBERTA_PATH +'vocab-roberta-base.json', 
    merges_file= ROBERTA_PATH +'merges-roberta-base.txt', 
    lowercase=True,
    add_prefix_space=True
)
SENTIMENT_ID = {'positive': 1313, 'negative': 2430, 'neutral': 7974}
AUTO = tf.data.experimental.AUTOTUNE

# Configuration

In [7]:
class Config():
    def __init__(self, config_path=None):

        if config_path:
            with open(config_path,'r') as json_file:
                self.__dict__ = json.load(json_file)
        else:
            
            self.version='tf-tpu-v1'

            # preprocessing
            self.max_len = 120
            # self.tweet_size_to_ignore = 1

            # learning
            self.drop_out = 0.3
            self.conv1d_size_1 = 128
            self.conv1d_size_2 = 64
            self.lr = 3e-5
            self.lr_decrease = 0.2
            self.label_smoothing = 0.25
            self.train_batch_size = 8
            self.valid_batch_size = 16
            self.num_epochs = 3
            self.num_folds = 5
            self.smart_span = True
            # dev
            self.num_folds_to_train = 1
            self.dev = True
            self.overwrite_model = True
        
        print(str(self.__dict__))

    def get(self, var):
        return getattr(self, var, None)


# config_file_name = "config-json-tpu-v28.txt"
# config_file_path = os.path.join(CHECKPOINT_PATH_HP_TUNING,config_file_name)
# if os.path.exists(config_file_path):
    # config = Config(config_file_path)
# else:
    # print("config file not found! Default configuration will be used")
config = Config()


{'version': 'tf-tpu-v1', 'max_len': 120, 'drop_out': 0.3, 'conv1d_size_1': 128, 'conv1d_size_2': 64, 'lr': 3e-05, 'lr_decrease': 0.2, 'label_smoothing': 0.25, 'train_batch_size': 8, 'valid_batch_size': 16, 'num_epochs': 3, 'num_folds': 5, 'smart_span': True, 'num_folds_to_train': 1, 'dev': True, 'overwrite_model': True}


# Utils

In [0]:
def print_to_file(config, text):
  try:
    f = open(CHECKPOINT_PATH + config.version + "_logs.txt", 'a')
  except:
    f = open(CHECKPOINT_PATH + config.version + "_logs.txt", 'w')

  print(text)
  f.write(time.strftime("%Y%m%d-%H%M%S"))
  f.write("  -- ")
  f.write(text)
  f.write("\n")

def jaccard(str1, str2): 

    a = set(str(str1).lower().split()) 
    b = set(str(str2).lower().split())

    c = a.intersection(b)

    if (len(a) + len(b) - len(c)) > 0:
      return float(len(c)) / (len(a) + len(b) - len(c))
    else:
      return 0

# Load Data

In [0]:
def import_data(config):
    def read_train():
        train=pd.read_csv(DATA_PATH + 'train.csv')
        train['text']=train['text'].astype(str)
        train['selected_text']=train['selected_text'].astype(str)

        return train

    def read_test():
        test=pd.read_csv(DATA_PATH + 'test.csv')
        test['text']=test['text'].astype(str)

        return test

    if (config.dev == True):
        train_df = read_train().sample(frac=.05, random_state=1).reset_index()
        test_df = read_test().sample(frac=.05, random_state=1).reset_index()
    else:
        train_df = read_train()
        test_df = read_test()


    print_to_file(config, "train_df shape:" + str(train_df.shape))
    print_to_file(config, "test_df shape:" + str(test_df.shape))
    
    return train_df, test_df

# Preprocess Data

In [0]:
def preprocess_train(config, train_df):
    ct = train_df.shape[0]
    input_ids = np.ones((ct,config.max_len),dtype='int32')
    attention_mask = np.zeros((ct,config.max_len),dtype='int32')
    token_type_ids = np.zeros((ct,config.max_len),dtype='int32')
    start_tokens = np.zeros((ct,config.max_len),dtype='int32')
    end_tokens = np.zeros((ct,config.max_len),dtype='int32')

    for k in range(train_df.shape[0]):
        
        # FIND OVERLAP
        text1 = " "+" ".join(train_df.loc[k,'text'].split())
        text2 = " ".join(train_df.loc[k,'selected_text'].split())
        idx = text1.find(text2)
        chars = np.zeros((len(text1)))
        chars[idx:idx+len(text2)]=1
        if text1[idx-1]==' ': chars[idx-1] = 1 
        enc = tokenizer.encode(text1) 
            
        # ID_OFFSETS
        offsets = []; idx=0
        for t in enc.ids:
            w = tokenizer.decode([t])
            offsets.append((idx,idx+len(w)))
            idx += len(w)
        
        # START END TOKENS
        toks = []
        for i,(a,b) in enumerate(offsets):
            sm = np.sum(chars[a:b])
            if sm>0: toks.append(i) 
            
        s_tok = SENTIMENT_ID[train_df.loc[k,'sentiment']]
        input_ids[k,:len(enc.ids)+5] = [0] + enc.ids + [2,2] + [s_tok] + [2]
        attention_mask[k,:len(enc.ids)+5] = 1
        if len(toks)>0:
            start_tokens[k,toks[0]+1] = 1
            end_tokens[k,toks[-1]+1] = 1
    return  input_ids, attention_mask, token_type_ids, start_tokens, end_tokens

In [0]:
def generate_dataset(config, idxT, idxV, input_ids, attention_mask, token_type_ids, start_tokens, end_tokens):
    
    # Trainset
    trn_input_ids = input_ids[idxT,]
    trn_att_mask = attention_mask[idxT,]
    trn_token_type_ids = token_type_ids[idxT,]
    
    trn_start_tokens = start_tokens[idxT,]
    trn_end_tokens = end_tokens[idxT,]
    
    # Validation set
    val_input_ids = input_ids[idxV,]
    val_att_mask = attention_mask[idxV,]
    val_token_type_ids = token_type_ids[idxV,]
    
    val_start_tokens = start_tokens[idxV,]
    val_end_tokens = end_tokens[idxV,]
    
    # Generating tf.data object
    train_dataset = (
        tf.data.Dataset
        .from_tensor_slices(({'input_ids':trn_input_ids, 'attention_mask': trn_att_mask, 'token_type_ids': trn_token_type_ids}, 
                             {'start_tokens': trn_start_tokens, 'end_tokens': trn_end_tokens}))
        .shuffle(2048)
        .batch(config.train_batch_size)
        .prefetch(AUTO)
    )
    
    valid_dataset = (
        tf.data.Dataset
        .from_tensor_slices(({'input_ids':val_input_ids, 'attention_mask': val_att_mask, 'token_type_ids': val_token_type_ids}, 
                             {'start_tokens': val_start_tokens, 'end_tokens': val_end_tokens}))
        .batch(config.valid_batch_size)
        .cache()
        .prefetch(AUTO)
    )
    
    return train_dataset, valid_dataset

# Design Model

In [0]:
def scheduler(epoch):
    return config.lr * config.lr_decrease**epoch

def build_model(config):
    ids = tf.keras.layers.Input((config.max_len,), dtype=tf.int32, name='input_ids')
    att = tf.keras.layers.Input((config.max_len,), dtype=tf.int32, name='attention_mask')
    tok = tf.keras.layers.Input((config.max_len,), dtype=tf.int32, name='token_type_ids')

    robert_config = RobertaConfig.from_pretrained(ROBERTA_PATH + 'config-roberta-base.json')
    bert_model = TFRobertaModel.from_pretrained(ROBERTA_PATH + 'pretrained-roberta-base.h5',config=robert_config)
    x = bert_model(ids,attention_mask=att,token_type_ids=tok)
    
    
    x1 = tf.keras.layers.Dropout(config.drop_out)(x[0]) 
    x1 = tf.keras.layers.Conv1D(config.conv1d_size_1, 2, padding='same')(x1)
    x1 = tf.keras.layers.LeakyReLU()(x1)
    x1 = tf.keras.layers.Conv1D(config.conv1d_size_2, 2, padding='same')(x1)
    x1 = tf.keras.layers.Dense(1)(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    x1 = tf.keras.layers.Activation('softmax', name='start_tokens')(x1)
    
    x2 = tf.keras.layers.Dropout(config.drop_out)(x[0]) 
    x2 = tf.keras.layers.Conv1D(config.conv1d_size_1, 2, padding='same')(x2)
    x2 = tf.keras.layers.LeakyReLU()(x2)
    x2 = tf.keras.layers.Conv1D(config.conv1d_size_2, 2, padding='same')(x2)
    x2 = tf.keras.layers.Dense(1)(x2)
    x2 = tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Activation('softmax', name='end_tokens')(x2)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1,x2])

    return model

# Training

In [0]:
def run_train(config, train_df, input_ids, attention_mask, token_type_ids, start_tokens, end_tokens):
    jac = []
    oof_start = np.zeros((input_ids.shape[0],config.max_len))
    oof_end = np.zeros((input_ids.shape[0],config.max_len))

    skf = StratifiedKFold(n_splits=config.num_folds, shuffle=True, random_state=777)
    for fold, (idxT,idxV) in enumerate(skf.split(input_ids, train_df.sentiment.values)):
        if fold < config.num_folds_to_train:
            print_to_file(config, '#'*25)
            print_to_file(config, '### FOLD %i'%(fold+1))
            print_to_file(config, '#'*25)

            trn_dataset, val_dataset = generate_dataset(config, idxT, idxV, input_ids, attention_mask, token_type_ids, start_tokens, end_tokens)

            if config.overwrite_model or (not os.path.exists(CHECKPOINT_PATH + '%s-roberta-%i.h5'%(config.version,fold))):
                K.clear_session()
                with strategy.scope():
                    model = build_model(config)
                    model.compile(
                        optimizer = tf.keras.optimizers.Adam(learning_rate=config.lr),
                        loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=config.label_smoothing)
                        )
            
                reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda epoch: config.lr * config.lr_decrease**epoch, verbose=1)

                sv = tf.keras.callbacks.ModelCheckpoint(
                    CHECKPOINT_PATH + '%s-roberta-%i.h5'%(config.version,fold), monitor='val_loss', verbose=1, save_best_only=True,
                    save_weights_only=True, mode='auto', save_freq='epoch')
                hist = model.fit(
                    trn_dataset,
                    epochs=config.num_epochs,
                    batch_size=config.train_batch_size,
                    verbose=1,
                    callbacks=[sv, reduce_lr],
                    validation_data=val_dataset
                    )
            else:
                print_to_file(config, "training Fold %d is skipped!"%(fold+1))

            print_to_file(config, 'Loading model...')
            model.load_weights(CHECKPOINT_PATH + '%s-roberta-%i.h5'%(config.version,fold))
            
            print_to_file(config, 'Predicting OOF...')
            oof_start[idxV,],oof_end[idxV,] = model.predict(val_dataset, verbose=1)
            
            # DISPLAY FOLD JACCARD
            all = []
            for k in idxV:
                a = np.argmax(oof_start[k,])
                b = np.argmax(oof_end[k,])

                if (a>b) and config.smart_span: 
                    max_len = len(oof_start[k,])
                    a = np.tile(oof_start[k,], (max_len, 1))
                    b = np.tile(oof_end[k,], (max_len, 1))
                    c = np.tril(a + b.T, k=0).T
                    c[c == 0] = -1000
                    a = np.unravel_index(c.argmax(), c.shape)[0]
                    b = np.unravel_index(c.argmax(), c.shape)[1]

                if (a>b):
                    st = train_df.loc[k,'text'] 
                else:
                    text1 = " "+" ".join(train_df.loc[k,'text'].split())
                    enc = tokenizer.encode(text1)
                    st = tokenizer.decode(enc.ids[a-1:b])
                
                train_df.loc[k,'selected_text_predicted'] = st
                train_df.loc[k,'jaccard'] = jaccard(train_df.loc[k,'selected_text_predicted'],train_df.loc[k,'selected_text'])
                all.append(train_df.loc[k,'jaccard'])

            jac.append(np.mean(all))
            print_to_file(config, '>>>> FOLD %i Jaccard = '%(fold+1) + str(np.mean(all)))


    print_to_file(config, 'Overall averrage of Jaccards in %i folds = '% (fold+1) + str(np.mean(jac)))
    return (np.mean(jac))

In [0]:
train_df, test_df = import_data(config)
input_ids, attention_mask, token_type_ids, start_tokens, end_tokens = preprocess_train(config, train_df)
run_train(config, train_df, input_ids, attention_mask, token_type_ids, start_tokens, end_tokens)

train_df shape:(1374, 5)
test_df shape:(177, 4)
#########################
### FOLD 1
#########################

Epoch 00001: LearningRateScheduler reducing learning rate to 3e-05.
Epoch 1/3


138/138 [==============================] - ETA: 0s - loss: 1.2614 - end_tokens_loss: 0.6307 - start_tokens_loss: 0.6307
Epoch 00001: val_loss improved from inf to 1.21963, saving model to /content/drive/My Drive/input/model_checkpoint/roberta/tf-tpu-v1-roberta-0.h5
138/138 [==============================] - 27s 199ms/step - loss: 1.2614 - end_tokens_loss: 0.6307 - start_tokens_loss: 0.6307 - val_loss: 1.2196 - val_end_tokens_loss: 0.6096 - val_start_tokens_loss: 0.6101 - lr: 3.0000e-05

Epoch 00002: LearningRateScheduler reducing learning rate to 6e-06.
Epoch 2/3
138/138 [==============================] - ETA: 0s - loss: 1.2575 - end_tokens_loss: 0.6287 - start_tokens_loss: 0.6288
Epoch 00002: val_loss improved from 1.21963 to 1.21935, saving model to /content/drive/My Drive/input/model_checkpoint/roberta/tf-tpu-v1-roberta-0.h5
138/138 [==============================] - 27s 198ms/step - loss: 1.2575 - end_tokens_loss: 0.6287 - start_tokens_loss: 0.6288 - val_loss: 1.2193 - val_end_toke

In [0]:
# HyperParameter Tunning
session_num = 0
for num_units1 in HP_NUM_UNITS1.domain.values:
  for num_units2 in HP_NUM_UNITS2.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
      for l2 in (HP_L2.domain.min_value, HP_L2.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
          
          hparams = {
              HP_NUM_UNITS1: num_units1,
              HP_NUM_UNITS2: num_units2,
              HP_DROPOUT: dropout_rate,
              HP_L2: l2,
              HP_OPTIMIZER: optimizer
              
          }
          run_name = "run-%d" % session_num
          print('--- Starting trial: %s' % run_name)
          print({h.name: hparams[h] for h in hparams})
          run('logs/hparam_tuning/' + run_name, hparams)
          session_num += 1
